In [14]:
import torch
print(torch.__version__)

2.9.0+cpu


### Inference using pipeline

In [15]:
from transformers import pipeline
import torch
pipe = pipeline("text-classification", model="ProsusAI/finbert")

Device set to use cpu


In [16]:
texts = [
    "Apple reported strong quarterly earnings today.",
    "NVIDIA just shared the stock dividend."
]

In [17]:
results = pipe(texts)
print(results)

[{'label': 'positive', 'score': 0.9521700739860535}, {'label': 'neutral', 'score': 0.9398248791694641}]


In [18]:
for text, result in zip(texts, results):
    print(f"Text: {text}")
    print(f"Prediction: {result["label"]}")
    print(f"Confidence: {result["score"]}\n")

Text: Apple reported strong quarterly earnings today.
Prediction: positive
Confidence: 0.9521700739860535

Text: NVIDIA just shared the stock dividend.
Prediction: neutral
Confidence: 0.9398248791694641



### Inference using Model and Tokenizer

In [19]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
print(device)

cpu


In [21]:
def predict_sentiment(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    inputs = {key: value.to(device) for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)

    predict = torch.nn.functional.softmax(outputs.logits, dim=1)

    labels = ["positive", "negative", "neutral"]

    results = []
    for i, text in enumerate(texts):
        probs = predict[i].cpu().numpy()
        predict_class = torch.argmax(predict[i]).item()

        results.append({
        "text": text,
        "label": labels[predict_class],
        "score": probs[predict_class],
        "scores": {label: float(prob) for label, prob in zip(labels, probs)}
        })

    return results

In [22]:
texts = [
    "Apple reported strong quarterly earnings today.",
    "NVIDIA just shared the stock dividend."
]

In [23]:
results = predict_sentiment(texts)
for result in results:
    print(f"Text: {result["text"]}")
    print(f"Prediction: {result["label"]}")
    print(f"Confidence: {result["score"]:.4f}")
    print(f"All scores: {result["scores"]}\n")

Text: Apple reported strong quarterly earnings today.
Prediction: positive
Confidence: 0.9522
All scores: {'positive': 0.9521700739860535, 'negative': 0.021184522658586502, 'neutral': 0.026645421981811523}

Text: NVIDIA just shared the stock dividend.
Prediction: neutral
Confidence: 0.9398
All scores: {'positive': 0.04332476481795311, 'negative': 0.016850389540195465, 'neutral': 0.9398248791694641}



### Inference using huggingface endpoint

In [24]:
from google.colab import userdata
hf_token = userdata.get('hf_token')

In [25]:
import os
from huggingface_hub import InferenceClient
os.environ['HF_TOKEN'] = hf_token
client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)

In [26]:
results = client.text_classification(
    "The stock market just got crashed.",
    model="ProsusAI/finbert",
)

for i, result in enumerate(results):
    print(f"  Predicted label: {result['label']}")
    print(f"  Confidence score: {result['score']:.4f}")
    print(f"  Confidence percentage: {result['score']*100:.2f}%\n")

  Predicted label: negative
  Confidence score: 0.7307
  Confidence percentage: 73.07%

  Predicted label: neutral
  Confidence score: 0.2418
  Confidence percentage: 24.18%

  Predicted label: positive
  Confidence score: 0.0275
  Confidence percentage: 2.75%

